In [3]:
import aiohttp
from models.siac import SiacAuth, SiacProofOfRegistration, SiacSubject, SiacLesson
from io import StringIO
from http.cookies import SimpleCookie
import pandas as pd
from fastapi.exceptions import HTTPException
from starlette.status import HTTP_400_BAD_REQUEST


async def auth_cookies(cpf: str, senha: str) -> SimpleCookie:
    login_url = "https://siac.ufba.br/SiacWWW/LogonSubmit.do"
    data = {"cpf": cpf, "senha": senha}
    headers = {"Content-Type": "application/x-www-form-urlencoded"}

    async with aiohttp.ClientSession() as session:
        async with session.post(login_url, data=data, headers=headers) as response:
            response_text = await response.text()
            
            if "O CPF informado não é válido." in response_text:
                raise HTTPException(
                    status_code=HTTP_400_BAD_REQUEST, 
                    detail="Invalid CPF."
                )
            if "For input string: " in response_text:
                raise HTTPException(
                    status_code=HTTP_400_BAD_REQUEST, 
                    detail="Invalid CPF."
                )
            if "A senha informada para o usuário de CPF" in response_text:
                raise HTTPException(
                    status_code=HTTP_400_BAD_REQUEST, 
                    detail="Invalid Password."
                )
            return response.cookies


In [4]:
cookies = await auth_cookies("85804437500", "$n!7Pro9MnQxtBHkzm9&&vCmPpV!8%669%ftGB")
comprovante_url = "https://siac.ufba.br/SiacWWW/ConsultarComprovanteMatricula.do"

async with aiohttp.ClientSession(cookies=cookies) as session:
    async with session.get(comprovante_url) as response:
        dfs = pd.read_html(StringIO(await response.text()))


'JONATHAN BISPO DOS SANTOS'